<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RAG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU


%pip install langchain_community -qU
%pip install langchainhub -qU

!pip install chromadb langchain-chroma -qU

%pip install langchain-chromadb -qU
%pip install pypdf -qU


# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic




# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

In [ ]:


!pip install unstructured
!pip install "unstructured[md]"

#
# En terminal o Colab
!python -m nltk.downloader punkt
!python -m nltk.downloader punkt_tab
!python -m nltk.downloader averaged_perceptron_tagger
!python -m nltk.downloader averaged_perceptron_tagger_eng
!python -m nltk.downloader universal_tagset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=e372e989afc169e9768fd326b8a4a0eeafc7bd60a9edeb0ec502ce367a4b334a
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk',

In [ ]:
from pathlib import Path
from itertools import chain
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# Ruta a tu carpeta de archivos Markdown
markdown_folder = "/content/"

# Cargar todos los archivos .md y .mdx
documents = []
for file_path in chain(Path(markdown_folder).glob("*.md"), Path(markdown_folder).glob("*.mdx")):
    loader = UnstructuredMarkdownLoader(str(file_path))
    documents.extend(loader.load())

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

# Dividir cada documento
splits = []
for doc in documents:
    splits.extend(markdown_splitter.split_text(doc.page_content))

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [ ]:

from langchain_community.vectorstores import Chroma

# Crear vector store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 10}  # recuperar los 5 documentos más relevantes
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.7)

# Prompt para generar respuestas
prompt = ChatPromptTemplate.from_template("""
Basándote en el siguiente contexto:
{context}

Responde a la siguiente pregunta:
{input}
""")

# Cadena de recuperación y generación usando LCEL
retrieval_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Consulta directa
response = retrieval_chain.invoke("Actua como experto en LangChain. Que es Fauna www.fauna.com ? Hay integracion en Langchain ? Pon un ejemplo")
print(response)

Fauna es una base de datos como servicio (DBaaS) que se centra en proporcionar un acceso escalable y flexible a los datos. Es conocida por su modelo de datos orientado a documentos y su capacidad para manejar datos en tiempo real, lo que la hace adecuada para aplicaciones modernas que requieren escalabilidad y una alta disponibilidad. Fauna ofrece características como transacciones ACID, soporte para múltiples modelos de datos (documentos, gráficos, etc.), y es particularmente útil en aplicaciones que utilizan arquitecturas sin servidor (serverless).

### Integración de Fauna en LangChain

Sí, hay una integración de Fauna en LangChain, que permite a los desarrolladores interactuar con la base de datos Fauna de manera sencilla dentro de sus aplicaciones construidas con LangChain.

### Ejemplo de uso de Fauna en LangChain

Para utilizar Fauna en LangChain, primero necesitamos instalar el paquete correspondiente. Aquí hay un ejemplo básico de cómo se podría utilizar Fauna en una aplicació